# Flux

M&I 4e

Ch. 16 Electric Potential 

- §16.3 &ndash; Potential Difference in a Uniform Field
- §16.5 &ndash; Potential Difference in a Non-Uniform Field

Ch. 21 Patterns of Fields in Space

- §21.2 &ndash; Definition of "Electric Flux"
- §21.3 &ndash; Gauss's Law

Ch. 22 Faraday's Law

- §22.2 &ndash; Faraday's Law

In [3]:
########
## Env Setup / Preamble
########
%load_ext autoreload
%autoreload complete 

%load_ext Cython

%display latex

################

import cython
from sage.all import *

################

import tempfile
import mmap
from multiprocessing import shared_memory

################

import numpy as np
import scipy.constants

np.random.seed( int(scipy.constants.golden_ratio * 1.e6) )

from numpy import vectorize
from numpy import r_, c_, s_

################

####    boiler-plate class for named constants
from sage.symbolic.constants import Constant
class ConstantFromExpression(Constant):
    def __init__(self, name, v):
        conversions = dict(maxima=repr(maxima.coerce(v)))
        Constant.__init__(self, name, conversions=conversions)
        self._v = v
    def __float__(self): return float(self._v)
    def __complex__(self): return complex(self._v)
    def _mpfr_(self, R): return R(self._v)
    def _real_double_(self, R): return R(self._v)

def NamedExpression(name, expr):
    return ConstantFromExpression(name,expr).expression()
####

import sage.symbolic.units as units     # <: TODO

π   = NamedExpression('π', scipy.constants.pi)          # pi
ε_0 = NamedExpression('ε_0', scipy.constants.epsilon_0) # electric permittivity of vacuum
μ_0 = NamedExpression('μ_0', scipy.constants.mu_0)      # magnetic permeability of vacuum

𝛋 = 1. / (4 * π * ε_0)      # Coulomb's constant in [ N·m²/C² ]
q = NamedExpression('q', -scipy.constants.elementary_charge)

ℎ_planck = NamedExpression('ℎ', scipy.constants.Planck)
ℏ = NamedExpression('ℏ', scipy.constants.hbar)

α = NamedExpression('α', scipy.constants.fine_structure)

m_e = NamedExpression('m_e', scipy.constants.electron_mass)
m_p = NamedExpression('m_p', scipy.constants.proton_mass)

################

Parallelism().set(nproc=8)
print(f"Python: {sys.version}")
version()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
Python: 3.12.5 (main, Feb 25 2025, 12:37:41) [Clang 16.0.0 (clang-1600.0.26.6)]


'SageMath version 10.6, Release Date: 2025-03-31'

In [4]:
##  sanity check
import this


---

<hr width="82%" style="border-top:2px dashed; border-bottom:none"/>

<hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 

<h2 id='1' style="display:none">[1]</h2>

>   ---
>   # [1] 
>
>   You move from location $i$ at <5, 5, 5> [m], to a location $f$ at $\langle 4, 10, 13\rangle$ [m].  
>
>   All along this path, there is a nearly uniform electric field 
>   $\mathbf{\vec{E}} = \langle 1200, 210, -500 \rangle$ [N/C].  
>
>   __Calculate $\Delta V = V_f - V_i$, including sign and units.__
>
>   &emsp;&emsp;&emsp;&emsp; $\Delta V = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>
>   __<p align='right'>(0.1 points)</p>__
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 

<h2 id='2' style="display:none">[2]*</h2>

>   ---
>   ## [2]*
>
>   A capacitor with a gap of 3 [mm] has a potential difference from one plate 
>   to the other of 34 [volts]. 
>
>   **What is the magnitude of the electric field between the plates?**
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{V}/\textrm{m}\right]$
>
>   __<p align='right'>(10 points)</p>__
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 

<h2 id='3' style="display:none">[3]*</h2>

>   ---
>   ## [3]*
>
>   
>
>   **Potential difference in a capacitor**
>
>   A capacitor consists of two large metal disks placed a distance $s$ apart 
>   (see figure above).
>
>   The radius $R$ of each disk is 7.8 [m], the gap $s$ between the disks is 
>   $1.2$ [mm], and the thickness of each disk is $0.4$ [mm]. 
>
>   The disk on the left has a net charge of $3.3 \times 10^{-4}$ [C], and the 
>   disk on the right has a net charge of $-3.3 \times 10^{-4}$ [C]. 
>
>   **Calculate the potential difference $V_2 - V_1$**, where:
>   - location 1 is inside the left disk at its center, and 
>   - location 2 is in the center of the air gap between the disks.
>
>   Use $\epsilon_0 = 8.85 \times 10^{-12}$ [C^2 / (N⋅m^2)]
>
>   &emsp;&emsp;&emsp;&emsp; $V_2 = V_1 = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>
>   <p align='right'><b>(10 points)</b></p>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='4' style="display:none">[4]*</h2>

>   ---
>   ## [4]*
>
>   Three charged metal disks are arranged as shown (cutaway view, see figure 
>   below). 
>
>   The disks are held apart by insulating supports not shown in the diagram. 
>   Each disk has an area of 2.3 [m²] (this is the area of one flat surface of 
>   the disk).
>
>   Use the values
>   - $\epsilon_0 = 8.85e10^{-12}$ [[C² / (N⋅m²)]],
>   - charge $Q_1 = 3 \times 10^{-8}$ Coulombs,
>   - charge $Q_2 = 2 \times 10^{-7}$ Coulombs.
>
>   
>
>   <p align='right'><b>(10 points)</b></p>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='4a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the magnitude of the electric field in the region between disks 1 and 2?__
>
>   &emsp;&emsp;&emsp;&emsp; $\lvert\mathbf{\vec{E}}_{12}\rvert = \fbox{\hspace{8em} ?}\ \left[\textrm{V}/\textrm{m}\right]$
>
>   __What is the direction of the electric field between disks 1 and 2?__ 
>
>   (select one):
>
>   - [ ]   $+\hat{x}$
>   - [ ]   $-\hat{x}$
>   - [ ]   $+\hat{y}$
>   - [ ]   $-\hat{y}$
>   - [ ]   $+\hat{z}$
>   - [ ]   $-\hat{z}$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='4b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(B)
>
>   __Which of the following statements are true?__  Choose all that apply. 
>
>   - [ ]   $V_B - V_A = - ((Q_1 / 2.3) / \epsilon_0) \cdot (0.003)\ \left[\textrm{V}\right]$
>   - [ ]   $V_B - V_A = 0$
>   - [ ]   $\mathbf{\vec{E}} \perp \Delta\vec{l}$, along a path from $A$ to $B$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='4c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/> 
>
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(C)
>
>   __To calculate $V_C - V_B$, which path should you choose?__ (select one):
>
>   - [ ]   A path starting at $B$ and ending at $C$.
>
>   - [ ]   It doesn't matter whether the path starts or ends at $B$.
>
>   - [ ]   A path starting at $C$ and ending at $B$.
>
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='4d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(D)
>
>   __The sign of $V_C - V_B$ should be:__ (select one):
>   
>   - [ ]   Zero - no sign. 
>
>   - [ ]   Positive, because $\Delta \vec{l}$ is opposite (anti-parallel) to the direction of $\mathbf{\vec{E}}$
>
>   - [ ]   Negative, because $\Delta \vec{l}$ is in the same direction (parallel) of $\mathbf{\vec{E}}$. 
>
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='4e' style="display:none">(E)</h2>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(E)
>
>   __What is the potential difference $V_C - V_B$?__
>
>   &emsp;&emsp;&emsp;&emsp; $V_C - V_B = \fbox{\hspace{8em} ?}\ \left[\textrm{V}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='4f' style="display:none">(F)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(F)
>
>   __What is the potential difference $V_D - V_C$?__
>
>   &emsp;&emsp;&emsp;&emsp; $V_D - V_C = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='4g' style="display:none">(G)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(G)
>
>   __What is the potential difference $V_F - V_D$?__
>
>   &emsp;&emsp;&emsp;&emsp; $V_F - V_D = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='4h' style="display:none">(H)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(H)
>
>   __What is the potential difference $V_G - V_F$?__
>
>   &emsp;&emsp;&emsp;&emsp; $V_G - V_F = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='4i' style="display:none">(I)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(I)
>
>   __What is the potential difference $V_G - V_A$?__
>
>   &emsp;&emsp;&emsp;&emsp; $V_G - V_A = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='4j' style="display:none">(J)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(J)
>
>   The charged disks have tiny holes which allow a particle to pass through them.  
>   An electron, which is traveling at a fast speed, approaches the plates from 
>   the left side.
>
>   It travels along a path from $A$ to $G$.
>
>   Since no external work is done on the system of plates + electron, 
>   $\Delta K + \Delta U = W_\textrm{ext} = 0$. 
>
>   Consider the following states:
>   - _Initial_, electron at location $A$.
>   - _Final_, electron at location $G$.
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h4 id='4j_i' style="display:none">{J.ⅰ}</h4>

>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>
>
>   #### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;{J.ⅰ}
>
>   __What is the change in the potential energy of the system?__
>
>   &emsp;&emsp;&emsp;&emsp; $\Delta U = \fbox{\hspace{8em}?}\ \left[\textrm{J}\right]$
>
>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<h4 id='4j_ii' style="display:none">{J.ⅱ}</h4>

>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>
>
>   #### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;{J.ⅱ}
>
>   __What is the change in kinetic energy of the electron?__
>
>   &emsp;&emsp;&emsp;&emsp; $\Delta K = \fbox{\hspace{8em}?}\ \left[\textrm{J}\right]$
>
>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='5' style="display:none">[5]*</h2>

>   ---
>   ## [5]*
>
>   The figure shows a disk-shaped region of radius 2 [cm], on which there is a 
>   uniform electric field, of magnitude 291 [V/m], at an angle of $\theta = 28$
>   degrees to the plane of the disk.
>
>   
>
>   __Calculate the electric flux on the disk, and include the correct units.__ (select one:)
>
>   &emsp;&emsp;&emsp;&emsp; $\textrm{flux} = \fbox{\hspace{8em}?}$
>
>   - [ ]   [V⋅m]
>   - [ ]   [V⋅m²]
>   - [ ]   [V/m²]
>   - [ ]   [V/m]
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='6' style="display:none">[6]</h2>

>   ---
>   ## [6]
>
>   The figure shows a box, on whose surfaces the electric field is measured to 
>   be horizontal and to the right. 
>
>   On the left face ($3$ [[cm]] by $2$ [[cm]]), the magnitude of the electric 
>   field $\mathbf{E}_1$ is $280$ volts/meter; and on the right face, the 
>   magnitude of the electric field $\mathbf{E}_2$ is 1190 volts/meter.
>
>   On the other faces, only the direction is known (horizontal for face $i$, 
>   $\mathbf{E}_{i \not \in \{1,2\}}$).
>
>   __Calculate the electric flux on every face of the box, the total flux, and the total amount of charge that is inside the box.__
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>   

<h3 id='6a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(A)
>
>   Electric Flux through the top face of the box is 
>   
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{V}\cdot\textrm{m}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='6b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(B)
>
>   Electric Flux through the bottom face of the box is 
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{V}\cdot\textrm{m}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='6c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(C)
>
>   Electric Flux through the left face of the box is 
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{V}\cdot\textrm{m}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='6d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(D)
>
>   Electric Flux through the right face of the box is 
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{V}\cdot\textrm{m}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='6e' style="display:none">(E)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(E)
>
>   Electric Flux through the front face of the box is 
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{V}\cdot\textrm{m}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='6f' style="display:none">(F)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(F)
>
>   Electric Flux through the back face of the box is 
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{V}\cdot\textrm{m}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='6g' style="display:none">(G)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(G)
>
>   Total Flux through all faces of the box is 
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{V}\cdot\textrm{m}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='6h' style="display:none">(H)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(H)
>
>   Electric Flux through the left face of the box is 
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{V}\cdot\textrm{m}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='6i' style="display:none">(I)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(I)
>
>   The total electric charge inside the box (recall that $\epsilon_0 = 8.85 \times 10^{-12}$ [C^2 / N⋅m^2]) is
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{C}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='7' style="display:none">[7]*</h2>

>   ---
>   ## [7]*
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   The electric field has been measured to be vertically upward everywhere on 
>   the surface of a box 20 [[cm]] long, 4 [[cm]] high, and 3 [[cm]] deep (shown
>   in the figure below). 
>
>   Over all the surface of the bottom face of the box, $\mathbf{\vec{E}}_1 = 1700$ [V/m], 
>   all over the surface of the side faces of the box $\mathbf{\vec{E}}_2 = 900$ [V/m], 
>   and over all the top surface of the box $\mathbf{\vec{E}}_3 = 800$ [V/m].
>
>   
>
>   __What is the amount of charge enclosed by the box?__ <br>
>   Use the accurate value $\epsilon_0 = 8.85 \times 10^{-12}$ [C^2 / N⋅m^2]
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{C}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(B)
>
>   The electric field is horizontal and has the values indicated on the surface 
>   of a cylinder shown in the figure. 
>
>   $\mathbf{\vec{E}}_1 = 1700$ [N/C], $\mathbf{\vec{E}}_2 = 1400$ [N/C], and 
>   $\mathbf{\vec{E}}_3 = 1100$ [N/C].
>
>   
>
>   __What is the amount of charge enclosed by the cylinder?__ <br>
>   Use the accurate value $\epsilon_0 = 8.85 \times 10^{-12}$ [C^2 / N⋅m^2]
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{C}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-top:2px dashed; border-bottom:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(C)
>
>   The electric field has been measured to be horizontal, and to the right, 
>   everywhere on the closed trapezoidal box shown in the figure below. 
>
>   Over all the surface of the left face of the trapezoidal box $\mathbf{\vec{E}}_1 = 90$ [V/m], 
>   and over all the surface of the slanting right face of the box $\mathbf{\vec{E}}_2 = 450$ [V/m].
>
>   On the surface of the top face of the box, the average field is $\mathbf{\vec{E}}_3 = 300$ [V/m], 
>   and on the surfaces of the front and back faces the average field is $\mathbf{\vec{E}}_4 = 175$ [V/m], 
>   and on the surface of the bottom face of the box the average field is $\mathbf{\vec{E}}_5 = 240$ [V/m].
>
>   
>
>   __How much charge is enclosed by the slanted trapezoidal box?__ <br>
>   Use the accurate value $\epsilon_0 = 8.85 \times 10^{-12}$ [C² / N⋅m²]
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{C}\right]$
>
>   <hr width="64%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='8' style="display:none">[8]*</h2>

>   ---
>   ## [8]*
>
>   The electric field has been measured to be horizontal, and to the right, 
>   everywhere on the closed box as shown in the figure below. 
>
>   
>
>   Over all the surface of the left side of the box $\mathbf{E}_1 = 107$ [V/m],
>   and all over the surface of the slanting right face of the box $\mathbf{E}_2 = 273$ [V/m].
>
>   On the top face, the average field is $\mathbf{E}_3 = 160$ [V], on the front
>   and back the average field is $\mathbf{E}_4 = 170$ [V/m], and on the bottom 
>   face the average field is $\mathbf{E}_5 = 190$ [V/m]. 
>
>   __How much charge is inside the box?__
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{C}\right]$
>
>   __Explain briefly:__ (select one):
>   
>   - [ ]   The charge inside the box is zero, because the total flux across all 
>           faces is zero. 
>
>   - [ ]   All six faces of the wedge have a non-zero flux. The sum of the flux
>           across all six faces gives the total flux, which can be used to find
>           the total charge. 
>
>   - [ ]   The charge inside the box is zero, because the flux across each 
>           individual face is zero. 
>
>   - [ ]   Only four faces of the wedge have a non-zero flux -- the front and 
>           back sides, and the top and bottom sides.  Their sum gives the total
>           flux, which can be used to find the total charge. 
>
>   - [ ]   Only two faces of the wedge have a non-zero flux -- the left side 
>           and the slanting side.  Their sum gives the total flux, which can be
>           used to find the total charge. 
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='9' style="display:none">[9]*</h2>

>   ---
>   ## [9]*
>
>   In the figure below the solenoid radius is $3$ [cm], and the ring radius is 
>   $14$ [cm]. 
>
>   $\mathbf{B} = 0.8$ [T] inside the solenoid, and $\mathbf{B} \approx 0$ outside the solenoid. 
>
>   
>
>   __What is the magnetic flux through the outer ring?__
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{T}\cdot\textrm{m}^2\right]$
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

---